In [1]:
# ─── PATH HACK FOR NOTEBOOK ───────────────────────────────────────────
import os, sys

# 1) assume the notebook lives in "notebooks/" and src/ is one level up:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root added to sys.path:", PROJECT_ROOT)
# ─────────────────────────────────────────────────────────────────────

Project root added to sys.path: c:\Users\nailf\Desktop\skripsi


In [2]:
import torch
from src.models.unet import UNet

model = UNet(n_channels=1, n_classes=1)
x = torch.randn(2, 1, 224, 224)       # batch of 2, 224×224
y = model(x)
print(y.shape)  # should be (2, 1, 224, 224)

torch.Size([2, 1, 224, 224])


In [3]:
import os, sys
from torch.utils.data import DataLoader
import torch, torch.nn as nn, torch.optim as optim
from src.data.loader import NpySegDataset
from src.models.unet import UNet
from tqdm.notebook import tqdm

# Absolute paths
PROJECT_ROOT   = os.path.abspath(os.path.join(os.getcwd(), ".."))
shenzhen_root  = os.path.join(PROJECT_ROOT, "data/processed/shenzhen")

# Datasets & Loaders
train_ds = NpySegDataset(shenzhen_root, split="train")
val_ds   = NpySegDataset(shenzhen_root, split="val")
train_dl = DataLoader(train_ds, batch_size=4, shuffle=True,  num_workers=2)
val_dl   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=2)

# Model + optimizer + loss
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = UNet(n_channels=1, n_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn   = nn.BCEWithLogitsLoss()

def dice_coef(pred, target, eps=1e-6):
    p = (pred > 0.5).float()
    t = target.float()
    intersect = (p * t).sum(dim=[1,2,3])
    union = p.sum(dim=[1,2,3]) + t.sum(dim=[1,2,3])
    return ((2*intersect + eps) / (union + eps)).mean().item()

# Training + validation loop for 1 epoch
model.train()
train_loss = 0.0
for imgs, masks in tqdm(train_dl, desc="Train", unit="batch"):
    imgs, masks = imgs.to(device), masks.to(device).float()
    preds       = model(imgs)
    loss        = loss_fn(preds, masks)
    optimizer.zero_grad(); loss.backward(); optimizer.step()
    train_loss += loss.item()
train_loss /= len(train_dl)

model.eval()
val_loss = 0.0
val_dice = 0.0
with torch.no_grad():
    for imgs, masks in tqdm(val_dl, desc="Validate", unit="batch"):
        imgs, masks = imgs.to(device), masks.to(device).float()
        preds       = model(imgs)
        val_loss   += loss_fn(preds, masks).item()
        val_dice   += dice_coef(torch.sigmoid(preds), masks)
val_loss /= len(val_dl)
val_dice /= len(val_dl)

print(f"\n✅ Epoch results — Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Dice: {val_dice:.4f}")


Train:   0%|          | 0/99 [00:00<?, ?batch/s]

Validate:   0%|          | 0/22 [00:00<?, ?batch/s]


✅ Epoch results — Train Loss: 0.1756 | Val Loss: 0.1059 | Val Dice: 0.9249
